# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [19]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
import matplotlib.pyplot as plt
# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
spaceship.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
#your code here
def fill_missing_values(spaceship):
    for column in spaceship.columns:
        if column in ["Name"]:
            continue
        if spaceship[column].dtype == 'float64':
            spaceship[column].fillna(spaceship[column].mean(), inplace=True)
        elif spaceship[column].dtype == 'object':
            spaceship[column].fillna(spaceship[column].mode()[0], inplace=True)
    
    # Special case for 'Name' column
    spaceship['Name'].fillna('J. D.', inplace=True)

    return spaceship

# Apply the function to DataFrame
spaceship = fill_missing_values(spaceship)

# Verify that there are no remaining missing values
print(spaceship.isnull().sum())

# Output the DataFrame to ensure correct replacements
spaceship.head()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [7]:
#your code here
spaceship['Cabin'] = spaceship['Cabin'].str[0]

In [8]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin         8693 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   bool   
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8693 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(3), float64(6), object(5)
memory usage: 772.6+ KB


In [9]:
#Drop the 'PassegnerId' and 'Name' columns
spaceship.drop(columns=['PassengerId', 'Name'], inplace=True)


In [10]:
#your code here
df_dummies = pd.get_dummies(spaceship, columns=spaceship.select_dtypes(include=['object']).columns, drop_first=True)

In [11]:
df_dummies.shape

(8693, 20)

In [12]:
#your code here
numerical_spaceship = df_dummies.select_dtypes(include=['float64','bool'])
numerical_spaceship

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,True,False,True,False,False,False,False,False,False,False,True
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,False,True,False,False,False,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,False,False,False,False,False,True
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,False,False,False,False,False,True
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,True,False,False,False,False,False,False,False,False,False,False
8689,True,18.0,False,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,False,True,False,True,False
8690,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,False,False,False,False,False,False,False,True,False,False,True
8691,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,True,False,False,False,False,True,False,False,False,False,False


In [53]:
numerical_spaceship.shape

(8693, 20)

In [13]:
#your code here
features = numerical_spaceship.drop(columns =["Transported"])
target = numerical_spaceship["Transported"]

In [14]:
#your code here
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [15]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [16]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [25]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [50]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor())

- Evaluate your model

In [28]:
pred = bagging_reg.predict(X_test_norm)

print("Bagging and Pasting MAE", mean_absolute_error(pred, y_test))
print("Bagging and Pasting RMSE", mean_squared_error(pred, y_test, squared=False))
print("Bagging and Pasting R2 score", bagging_reg.score(X_test_norm, y_test))

Bagging and Pasting MAE 0.28510749601886204
Bagging and Pasting RMSE 0.37534064904954223
Bagging and Pasting R2 score 0.436446095033742


# GRID SEARCH

- Define hyperparameters to fine tune.

In [42]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [43]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor())

In [44]:
model = GridSearchCV(estimator = bagging_reg, param_grid = grid, cv=5)

- Run Grid Search

In [45]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

In [46]:
model.best_params_

{'estimator__max_depth': 50,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 500}

In [47]:
best_model = model.best_estimator_

- Evaluate your model

In [52]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.2719899462458835
RMSE 0.3712797588500949
R2 score 0.44857454680881725


# RANDOM SEARCH

In [33]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
        "estimator__max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
        "estimator__max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

In [34]:
model = RandomizedSearchCV(estimator = bagging_reg, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)

In [35]:
model.fit(X_train_norm, y_train)

RandomizedSearchCV(cv=5,
                   estimator=BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                                              max_samples=1000,
                                              n_estimators=100),
                   n_jobs=-1,
                   param_distributions={'estimator__max_depth': [10, 20, 30, 40,
                                                                 50, 60, 70, 80,
                                                                 90, 100, 110],
                                        'estimator__max_leaf_nodes': [500, 777,
                                                                      1055,
                                                                      1333,
                                                                      1611,
                                                                      1888,
                                                                      2166,
                                                                      2444,
                                                                      2722,
                                                                      3000],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [36]:
model.best_params_

{'n_estimators': 1400,
 'estimator__max_leaf_nodes': 2444,
 'estimator__max_depth': 50}

In [37]:
best_model = model.best_estimator_

In [38]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.28439301751535795
RMSE 0.37354683281665696
R2 score 0.44181986142764695
